In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
missing_values = ["NA", "na", "?","-5","-1","MISSINGVAL","MISSINGVALUE","MISSEDDATA","???",""," "]
df1 = pd.read_csv("TrainData/Train.csv", na_values = missing_values)
df1_claim = pd.read_csv("TrainData/Train_Claim.csv",na_values = missing_values)
df1_demographics = pd.read_csv("TrainData/Train_Demographics.csv",na_values = missing_values)
df1_policy = pd.read_csv("TrainData/Train_Policy.csv",na_values = missing_values)
df1_vehicle = pd.read_csv("TrainData/Train_Vehicle.csv",na_values = missing_values)

In [3]:
df1_merge = df1.merge(df1_claim,on='CustomerID').merge(df1_demographics,on='CustomerID').merge(df1_policy,on ='CustomerID')

In [4]:
print(df1_merge.shape)
df1_merge.head()

(28836, 38)


,CustomerID,ReportedFraud,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,...,Country,InsurancePolicyNumber,CustomerLoyaltyPeriod,DateOfPolicyCoverage,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship
0,Cust20065,N,2015-01-09,Multi-vehicle Collision,Rear Collision,Total Loss,Other,State9,City4,Location 1404,...,India,129186,432,1998-11-14,State2,250/500,2000,1142.87,0,own-child
1,Cust37589,N,2015-02-20,Single Vehicle Collision,Side Collision,Minor Damage,Ambulance,State4,City3,Location 1675,...,India,146710,128,2000-08-19,State2,100/300,1000,1274.38,2857344,own-child
2,Cust24312,N,2015-01-14,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,State5,City2,Location 1546,...,India,133433,346,2000-06-23,State1,500/1000,745,1269.93,0,other-relative
3,Cust5493,Y,2015-01-07,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,State7,City4,Location 1413,...,India,114614,42,1992-01-15,State2,100/300,986,1218.60,0,other-relative
4,Cust7704,Y,2015-02-26,Multi-vehicle Collision,Side Collision,Major Damage,Other,State4,City6,Location 1367,...,India,116825,109,2001-09-23,State3,250/500,576,1431.02,4235779,unmarried


In [5]:
df1_vehicle_pivot = pd.pivot_table(df1_vehicle, values='VehicleAttributeDetails', index=['CustomerID'],
                         columns=['VehicleAttribute'], aggfunc=np.sum)

df1_vehicle_pivot.head()

VehicleAttribute,VehicleID,VehicleMake,VehicleModel,VehicleYOM
CustomerID,,,,
Cust10000,Vehicle26917,Audi,A5,2008
Cust10001,Vehicle15893,Audi,A5,2006
Cust10002,Vehicle5152,Volkswagen,Jetta,1999
Cust10003,Vehicle37363,Volkswagen,Jetta,2003
Cust10004,Vehicle28633,Toyota,CRV,2010


In [6]:
df1_final = df1_merge.merge(df1_vehicle_pivot,on='CustomerID')
print(df1_final.shape)
df1_final.head()

(28836, 42)


,CustomerID,ReportedFraud,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,...,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,VehicleID,VehicleMake,VehicleModel,VehicleYOM
0,Cust20065,N,2015-01-09,Multi-vehicle Collision,Rear Collision,Total Loss,Other,State9,City4,Location 1404,...,State2,250/500,2000,1142.87,0,own-child,Vehicle30662,Mercedes,E400,2005
1,Cust37589,N,2015-02-20,Single Vehicle Collision,Side Collision,Minor Damage,Ambulance,State4,City3,Location 1675,...,State2,100/300,1000,1274.38,2857344,own-child,Vehicle17183,Toyota,Highlander,2010
2,Cust24312,N,2015-01-14,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,State5,City2,Location 1546,...,State1,500/1000,745,1269.93,0,other-relative,Vehicle33982,Volkswagen,Passat,2002
3,Cust5493,Y,2015-01-07,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,State7,City4,Location 1413,...,State2,100/300,986,1218.60,0,other-relative,Vehicle12299,Toyota,Highlander,2011
4,Cust7704,Y,2015-02-26,Multi-vehicle Collision,Side Collision,Major Damage,Other,State4,City6,Location 1367,...,State3,250/500,576,1431.02,4235779,unmarried,Vehicle30447,Ford,Wrangler,2010


In [7]:
report_fraud = {'Y':1 , 'N':0}
df1_final.ReportedFraud = [report_fraud[i] for i in df1_final.ReportedFraud]

In [8]:
#df1_final.head(5)

In [9]:
#df1_final.isnull().sum()

In [10]:
print("Unique values in Insured Gender column ",df1_final['InsuredGender'].unique())
print("Unique values in Country column ",df1_final['Country'].unique())

df1_final['InsuredGender'] = df1_final['InsuredGender'].replace(np.nan,'FEMALE')
df1_final['Country'] = df1_final['Country'].replace(np.nan,'India')

Unique values in Insured Gender column  ['MALE' 'FEMALE' nan]
Unique values in Country column  ['India' nan]


In [11]:
df1_final['AmountOfTotalClaim'].fillna(df1_final['AmountOfTotalClaim'].mean(), inplace=True)
df1_final['PolicyAnnualPremium'].fillna(df1_final['PolicyAnnualPremium'].mean(), inplace=True)
df1_final['Witnesses'].fillna(df1_final['Witnesses'].mean(), inplace=True)
df1_final['IncidentTime'].fillna(df1_final['IncidentTime'].mean(), inplace=True)

In [12]:
df1_final['IncidentTime']=df1_final['IncidentTime'].round(1)

In [13]:
# Dropping columns which are having more than 15% of NA Values
df1_final.drop('PropertyDamage',axis=1,inplace=True)
df1_final.drop('PoliceReport',axis=1,inplace=True)
df1_final.drop('TypeOfCollission',axis=1,inplace=True)

In [14]:
#lets do Lable enconding coding to make more features 
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in df1_final:
    if df1_final[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(df1_final[col].unique())) <= 2:
            # Train on the training data
            le.fit(df1_final[col])
            # Transform both training and testing data
            df1_final[col] = le.transform(df1_final[col])
            
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

2 columns were label encoded.


In [15]:
# droping columns based on above result
df1_final.drop(['VehicleID','DateOfPolicyCoverage','IncidentAddress','DateOfIncident','VehicleModel','VehicleYOM','InsuredHobbies','VehicleMake','Policy_CombinedSingleLimit',
                'IncidentCity','InsuredEducationLevel','IncidentState','InsuredRelationship','InsurancePolicyState','IncidentTime','TypeOfIncident',
               'BodilyInjuries','Witnesses','InsuredZipCode','InsuredGender','Country','InsurancePolicyNumber','CustomerLoyaltyPeriod',
               'CapitalGains','CapitalLoss','CustomerID','InsuredOccupation','UmbrellaLimit','AuthoritiesContacted'],axis=1,inplace=True)

df1_final.head(10)

,ReportedFraud,SeverityOfIncident,NumberOfVehicles,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,InsuredAge,Policy_Deductible,PolicyAnnualPremium
0,0,Total Loss,3,43973.0,9396,4698,29879,58,2000,1142.87
1,0,Minor Damage,1,62310.0,6539,6539,49232,34,1000,1274.38
2,0,Major Damage,3,42824.0,6069,6069,30686,52,745,1269.93
3,1,Major Damage,3,45672.0,253,5741,39678,25,986,1218.60
4,1,Major Damage,3,81472.0,7407,14813,59252,27,576,1431.02
5,0,Major Damage,3,55504.0,5453,10907,39144,41,2000,1428.18
6,1,Minor Damage,1,47366.0,4306,8612,34448,20,1090,1147.96
7,0,Major Damage,3,53292.0,9902,9902,33488,38,2000,1250.79
8,0,Total Loss,2,59742.0,6638,6638,46466,22,1000,1283.29
9,0,Total Loss,3,46225.0,7886,7886,30453,37,1000,1125.05


In [16]:
df1_final.shape

(28836, 10)

In [17]:
df_new = df1_final

In [18]:
a = pd.get_dummies(df_new['SeverityOfIncident'],drop_first=True)
df_final = pd.concat([df_new,a],axis=1)

# b = pd.get_dummies(concat['AuthoritiesContacted'],drop_first=True)
# df_final = pd.concat([concat,b],axis=1)

# c = pd.get_dummies(concat1['InsuredOccupation'],drop_first=True)
# df_final = pd.concat([concat1,c],axis=1)

df_final.head()

,ReportedFraud,SeverityOfIncident,NumberOfVehicles,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,InsuredAge,Policy_Deductible,PolicyAnnualPremium,Minor Damage,Total Loss,Trivial Damage
0,0,Total Loss,3,43973.0,9396,4698,29879,58,2000,1142.87,0,1,0
1,0,Minor Damage,1,62310.0,6539,6539,49232,34,1000,1274.38,1,0,0
2,0,Major Damage,3,42824.0,6069,6069,30686,52,745,1269.93,0,0,0
3,1,Major Damage,3,45672.0,253,5741,39678,25,986,1218.60,0,0,0
4,1,Major Damage,3,81472.0,7407,14813,59252,27,576,1431.02,0,0,0


In [19]:
df_final.drop(['SeverityOfIncident'] ,axis=1,inplace=True)

# Splitting the data

In [27]:
print(df_final.columns)
print(df_final.shape)
print(df_final.dtypes)

Index(['ReportedFraud', 'NumberOfVehicles', 'AmountOfTotalClaim',
       'AmountOfInjuryClaim', 'AmountOfPropertyClaim', 'AmountOfVehicleDamage',
       'InsuredAge', 'Policy_Deductible', 'PolicyAnnualPremium',
       'Minor Damage', 'Total Loss', 'Trivial Damage'],
      dtype='object')
(28836, 12)
ReportedFraud              int64
NumberOfVehicles           int64
AmountOfTotalClaim       float64
AmountOfInjuryClaim        int64
AmountOfPropertyClaim      int64
AmountOfVehicleDamage      int64
InsuredAge                 int64
Policy_Deductible          int64
PolicyAnnualPremium      float64
Minor Damage               uint8
Total Loss                 uint8
Trivial Damage             uint8
dtype: object


In [21]:
X= df_final.drop('ReportedFraud',axis=1)
y= df_final['ReportedFraud']

In [22]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 11)

In [23]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
#Test_df_final = sc.transform(Test_df_final)

# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 108)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_val)

In [24]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)
cm

array([[3136, 1060],
       [ 577,  995]])

In [25]:
# Saving model to disk
pickle.dump(classifier, open('rfc.pkl','wb'))

# Loading model to compare the results
model = pickle.load(open('rfc.pkl','rb'))

In [26]:
unpickled_df = pd.read_pickle("rfc.pkl")
unpickled_df

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=108,
                       verbose=0, warm_start=False)

In [ ]:
#predictions = pd.DataFrame(y_pred)
#submission =Test_df1.join(predictions)
#submission.rename(columns = {0: "ReportedFraud"},inplace=True)
#submission_1.to_csv("E:/Varun/INSOFE/PhD_INSOFE/submission_NB.csv",index= False)
#----
# Pred_test = model_o.predict(Test_df_final)
# predictions_test = pd.DataFrame(Pred_test)
# submission_2 =Test_df1.join(predictions_test)